### Area Comparison
This notebook is the second notebook in the resourceAssessment workflow. 

In this notebook, we generate a grid cell by grid cell comparison of total developable area and percent developable area for a given resource type (wind, solar) and grid geoemtry (cesm2, era5). The inputs for are contained in **resourceAssessmentResults.gdb**. The outputs are saved to **resourceAssessmentResults.gdb**. 

**Note:** The user must run this notebook with the appropriate inputs each time they wish to generate an individual area comparison.

In [156]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

### Set up Join by Attribute

In [157]:
# Set input and output folder, create output file geodatabase

mainInputFolder = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data" # enter path to input folder
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\resourceAssessment\\analysis\\data\\" # enter path to output folder

In [158]:
# Set environment workspace to output workspace
env.workspace = os.path.join(mainOutputFolder, "resourceAssessmentResultsWECC.gdb") 

# Set name of output feature class
output_fc = "SL1_Wind_era5_areaAssessment_WECC"

In [159]:
# SET THESE VARIABLES
target_fc = "CandidateProjectArea_Wind_SL1_WU_projected_clipped_dissolved_wecc_intersect_era5_base_grid_point_joined" # ResourceAssessment to be compared
join_fc = "InclusionArea_Wind_SL1_ZRM_filters_dissolved_wecc_intersect_era5_base_grid_point_joined" # Resource Assessment to be compared
copy_fc = target_fc + "_copied" # Leave this 

In [160]:
# Define the target and join field names
target_field = "OBJECTID"
join_field = "OBJECTID"

In [ ]:
# Copy target_fc to copy_fc
arcpy.management.CopyFeatures(target_fc, copy_fc)

In [ ]:
# Perform the join using JoinField
arcpy.management.JoinField(copy_fc, target_field, join_fc, join_field)


In [ ]:
# Copy the joined feature class to the output feature class
arcpy.management.CopyFeatures(copy_fc, output_fc)

In [ ]:
# Delete the copied target feature class
arcpy.management.Delete(copy_fc)

### Add Percent Difference attribute to compare the two models

In [ ]:
# View the attributes the new feature classes
field_names = [field.name for field in arcpy.ListFields(output_fc)]
field_names

In [ ]:
# Add a 'pct_dif' field to the joined
arcpy.AddField_management(output_fc, "pct_dif", "DOUBLE")

# Calculate the area for all polygons and populate the "pct" field
num1 = 'IA_Area'
num2 = 'CPA_Area'
denom = 'Shape_Area'
expression = "None if (!{0}! is None or !{1}! is None or !{1}! == 0) else round((!{0}! - !{1}!) / !{2}!, 4)".format(
            num1, num2, denom)
arcpy.CalculateField_management(output_fc, "pct_dif", expression, "PYTHON")

### Clean up Attribute Table

In [167]:
# Define attributes we want to keep
attributes_to_keep = ["OBJECTID","Shape", "CPA_Area", "area_pct", "IA_Area", "IA_pct", "pct_dif", "Shape_Length","Shape_Area"] # change area_pct to CPA_pct

In [168]:
# Get a list of all field names in the output feature class
all_fields = [field.name for field in arcpy.ListFields(output_fc)]

In [169]:
# Determine the fields to delete (all except the specified attributes to keep)
fields_to_delete = [field for field in all_fields if field not in attributes_to_keep]

In [ ]:
# Delete the unwanted fields
arcpy.management.DeleteField(output_fc, fields_to_delete)

In [ ]:
# List new field names
new_field_names = [field.name for field in arcpy.ListFields(output_fc)]
new_field_names

### Reproject to WGS 1984 and Export

In [172]:
# Get epsg code for output_fc
epsg = arcpy.Describe(output_fc).spatialReference.factoryCode
epsg

102039

In [173]:
# Define the spatial reference of the desired output coordinate system
input_coordinate_system = arcpy.SpatialReference(epsg)
output_coordinate_system = arcpy.SpatialReference(4326)  # Example: WGS 84

In [ ]:
# Get the list of available datum transformations
transformations = arcpy.ListTransformations(input_coordinate_system, output_coordinate_system)

# Get best datum transformation
datum_transform = transformations[0]
datum_transform

In [175]:
# Define the path for the new folder to save the shapefile
output_folder = os.path.join(mainInputFolder, "shapefiles")

# Create the output folder if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [176]:
# Define the path and name for the new reprojected shapefile
reprojected_fc = os.path.join(output_folder, output_fc + "_reproject.shp")

In [ ]:
# Perform the reprojection and save as a shapefile
arcpy.management.Project(output_fc, reprojected_fc, output_coordinate_system, datum_transform)

In the next notebook, we will import our shapefiles into postGIS for processing, and export the data as .CSVs